In [31]:
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import json
import random

import pandas as pd
from tqdm import tqdm
import os

In [6]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# chat_model = ChatOpenAI(api_key=OPENAI_API_KEY)
client = OpenAI(api_key=OPENAI_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index('receipe')

In [3]:
# response = client.embeddings.create(
#     input="hello",
#     model="text-embedding-ada-002"
# )

# print(response.data[0].embedding)

[-0.025150660425424576, -0.019543994218111038, -0.027953993529081345, -0.03099534474313259, -0.024740738794207573, 0.027583742514252663, -0.012634835205972195, -0.008449669927358627, -0.017613396048545837, -0.008284378796815872, 0.03250279650092125, 0.004304174333810806, -0.024582060053944588, -0.0006231465958990157, 0.014135675504803658, -0.0016231564804911613, 0.03943179175257683, 0.0021504343021661043, 0.027041587978601456, -0.012489378452301025, -0.021091116592288017, 0.008965377695858479, 0.00839016493409872, -0.003259536111727357, -0.0053917886689305305, -0.00962654035538435, 0.011219944804906845, -0.0015396844828501344, 0.003557059681043029, -0.023286178708076477, 0.006783537566661835, -0.007854622788727283, -0.023907672613859177, -0.008912484161555767, 0.006862877402454615, -0.013778647407889366, 0.009533978067338467, -0.01420179195702076, 0.021990299224853516, -0.01057861652225256, 0.0033884630538523197, -0.014664607122540474, 0.00538187101483345, -0.014862955547869205, -0.031

In [3]:
proc = pd.read_csv('./data/whole_processed.csv')
proc.head()

,Recipe_ID,Recipe_Title,Dish_Name,Cooking_Method,Cooking_Status,Ingredient_List,Dish_Type,Cooking_Intro,Ingredient_Details,Serving_Size,Difficulty_Level,Cooking_Time
0,128671,어묵김말이,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내
2,128932,입안에서 톡톡톡,알밥,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내
3,131871,★현미호두죽,현미호두죽,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내


In [13]:
proc10 = proc.iloc[:100, :]
proc1000 = proc.iloc[100:1000, :]

In [14]:
list_without_recipe_id = proc1000.drop(columns=['Recipe_ID']).apply(lambda row: ', '.join([f'{col}: {val}' for col, val in row.items()]), axis=1).tolist()
list_without_recipe_id

['Recipe_Title: ◈ 겹속에 숨어있는 소세지 - 소세지 크로와상, Dish_Name: 소세지크로와상, Cooking_Method: 굽기, Cooking_Status: 간식, Ingredient_List: 밀가루, Dish_Type: 빵, Cooking_Intro: 그냥 크로와상이 아니고 소세지가 들어있어서 식사대용으로도 든든하더군요. 집에서 만들기 힘들어보이긴 하지만 한번쯤 특식으로 만들어봐도 좋을듯해요., Ingredient_Details: [재료] 강력분 360g| 박력분 90g| 물 200g| 드라이 이스트 11g| 소금 9g| 설탕 67g| 버터 45g| 탈지분유 13g| 달걀 1개| 충전용 버터 350g| 달걀 물 약간, Serving_Size: 6인분이상, Difficulty_Level: 중급, Cooking_Time: 2시간이내',
 'Recipe_Title: 씹을수록 고소하고 든든한 유기농 뮈슬리바, Dish_Name: 뮈슬리바, Cooking_Method: 끓이기, Cooking_Status: 간식, Ingredient_List: 곡류, Dish_Type: 디저트, Cooking_Intro: 쿠키에 넣어 구워도 좋고  우유에 타서 먹어도 좋은 뮈슬리. 요 뮈슬리로 뮈슬리바를 만들어 놓으면입이 심심할때나 출출할 때 좋길래 만들어 보았어요. 만드는 방법도 초간하답니다., Ingredient_Details: [재료] 유기농 뮈슬리 250g| 버터 25g| 요리당 50g| 꿀 50g| 호박씨 50g, Serving_Size: 6인분이상, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 그린티 현미쉬폰, Dish_Name: 그린티현미쉬폰, Cooking_Method: 굽기, Cooking_Status: 간식, Ingredient_List: 밀가루, Dish_Type: 빵, Cooking_Intro: 100% 리얼 생크림  국내산 현미가루  일본산 말차  포피시드가 들어간 먹으면 건강해

In [15]:
# --------------------------------=
# |레시피 임베딩 & vectorDB에 업로드|
# ---------------------------------

# embeddings = []
# for i in tqdm(list_without_recipe_id):
#     response = client.embeddings.create(
#         input=i,
#         model="text-embedding-ada-002"
#     )
#     embeddings.append(response.data[0].embedding)

# # embeddings

# ids = proc1000['Recipe_ID'].to_list()
# # ids

# meta_datas = [{'text': text} for text in list_without_recipe_id]
# # meta_datas

# records = [{'id': str(i), 'values': j, 'metadata': k} for i,j,k in zip(ids, embeddings, meta_datas)]
# records

# index.upsert(vectors=records)

100%|██████████| 900/900 [05:27<00:00,  2.74it/s]


PineconeException: UNKNOWN:Error received from peer  {created_time:"2024-08-18T06:36:17.6136198+00:00", grpc_status:11, grpc_message:"Error, message length too large: found 6159445 bytes, the limit is: 4194304 bytes"}

In [33]:
index.upsert(
    vectors=[
        {"id": "vec5", "values": tmp}
    ],
    namespace="health"
)

upserted_count: 1

In [33]:
# index.upsert(vectors=records[:100])
# index.upsert(vectors=records[100:200])
# index.upsert(vectors=records[200:300])
# index.upsert(vectors=records[300:400])
# index.upsert(vectors=records[400:500])
# index.upsert(vectors=records[500:600])
# index.upsert(vectors=records[600:700])
# index.upsert(vectors=records[700:800])
# index.upsert(vectors=records[800:900])

upserted_count: 100

In [21]:
health = pd.read_csv('../health.csv')
health = health.to_string()
health

"                 질병명                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    식사요법의 필요성                      

In [9]:
# vectorDB에 검색하는 함수
def request_query(query):
    response = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002"
    )

    embedded_query=response.data[0].embedding
    results=index.query(embedded_query, top_k=3, include_metadata=True)

    return results['matches']

In [12]:
user_need = '뜨끈한 해물이 포함된 국물이 있는 요리.'
ingredients = '오징어| 새우| 어묵| 조개| 홍합| 고추가루'
disease = '당뇨병'

In [ ]:
# 100개 기준
# 임베딩: 2초
# GPT 응답속도: 12.3초 --> CHAT GPT UI 적용 가능?

In [11]:
pinecone_result = request_query(f'{user_need}, Ingredient_Details: {ingredients}')
recipes = [i['metadata']['text'] for i in pinecone_result]
recipes

['Recipe_Title: 국물이 깔끔한 홍합미역국, Dish_Name: 홍합미역국, Cooking_Method: 끓이기, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 국/탕, Cooking_Intro: 국물이 깔끔한 홍합미역국입니다., Ingredient_Details: [재료] 홍합 1/2줌| 물기짠 미역 1줌 [양념] 다진 마늘 2큰술| 국간장 3큰술| 소금 약간| 후추 약간, Serving_Size: 2인분, Difficulty_Level: 초급, Cooking_Time: 30분이내',
 'Recipe_Title: 떡국 골라먹는 재미와 함께 매콤 매콤 찡어 오징어 볶음~, Dish_Name: 오징어볶음, Cooking_Method: 볶음, Cooking_Status: 일상, Ingredient_List: 해물류, Dish_Type: 메인반찬, Cooking_Intro: 설 명절 잘 보내시고 복 많이 받으셨나요?? 강원도 원주서 대구로 오르락 내리락~ 정신없는 설 연휴가 자나고 나니 이제 한시름 놓아요~^^ 이것저것 설 음식으로 냉장고가 그득합니다.. 느끼한 설 음식에 질렸을 즈음~ 오징어 초 고추장 찍어 먹고 남은 오징어 이용 매콤 오징어 볶음 해 드셔보세요~ 느끼해진 입맛을 살려준답니다..^^ 함께 만들어 밥이나 소면에 비벼 먹어요~, Ingredient_Details: nan, Serving_Size: 4인분, Difficulty_Level: 아무나, Cooking_Time: 60분이내',
 'Recipe_Title: 뜨끈한 국물인 일품인 칼국수, Dish_Name: 칼국수, Cooking_Method: 끓이기, Cooking_Status: 일상, Ingredient_List: 밀가루, Dish_Type: 면/만두, Cooking_Intro: 으슬으슬 추울땐 뜨끈한 칼국수~, Ingredient_Details: [재료] 칼국수 3인분| 감자 1/2개| 애호박 1/3개| 계란 1개| 새우 1줌| 표고

In [13]:
# stream = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[{"role": "user", "content": "Say this is a test"}],
#     stream=True,
# )
# for chunk in stream:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")

This is a test. How can I assist you further?

In [22]:
system_prompt = """
당신은 한식 최고의 요리사이자 가정의학과 교수입니다. 
추천 요리 세 가지를 활용하여 레시피를 알려주고, 환자의 질병에 추천하는 순으로 순위 및 점수를 알려주세요.
"""

user_prompt = f"""
1. 추천요리1, 추천요리2, 추천요리3을 활용하여 레시피를 생성하세요.
2. 손님의 질병 정보와 식이요법 백과사전을 참고하여 각 레시피별 스코어를 정하세요.
3. 스코어가 높은 순으로 레시피를 말해주세요.

---
- 추천요리1: {recipes[0]}
- 추천요리2: {recipes[1]}
- 추천요리3: {recipes[2]}

---
- 식이요법 백과사전: {health}
- 질병정보: {disease}
"""



In [25]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-turbo-preview in organization org-tCAIJLieoZ5a5hHAL85SpD2O on tokens per min (TPM): Limit 30000, Requested 102125. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [10]:
print(response.choices[0].message.content)

여기 세 가지의 한식 요리 레시피를 추천해드릴게요. 오늘의 메뉴는 매콤 오징어 볶음우동, 해물을 가득 넣은 뚝배기 해물떡볶이, 그리고 새우볶음밥 고로케입니다. 각각의 요리 방법과 필요한 재료들을 자세히 알려드립니다.

### 1. 매콤 오징어 볶음우동
#### 재료
- 오징어 1마리
- 양파 1/4개
- 대파 1/2쪽
- 당근 1/3개
- 청고추 1개
- 홍고추 1개

#### 양념
- 고추장 1T
- 고춧가루 1T
- 다진 마늘 1T
- 참기름 1T
- 설탕 1t
- 간장 1t
- 청주 1t
- 후추 약간
- 소금 약간

#### 요리 방법
1. 오징어는 손질한 후 먹기 좋은 크기로 썰어주세요.
2. 양파, 대파, 당근, 청고추, 홍고추는 채 썰어줍니다.
3. 팬에 참기름을 두르고 다진 마늘을 넣어 향을 내줍니다.
4. 이어서 손질한 오징어를 넣어 볶아주세요.
5. 오징어가 반쯤 익었을 때 채 썬 야채들을 넣어 함께 볶아줍니다.
6. 고추장, 고춧가루, 설탕, 간장, 청주, 후추, 소금을 넣어 간을 맞추고 잘 섞어줍니다.
7. 우동 사리를 끓는 물에 잠깐 데친 후 오징어 볶음에 넣어 섞어줍니다.
8. 모든 재료가 잘 어우러지도록 볶아주면 완성입니다.

### 2. 해물을 가득 넣은 뚝배기 해물떡볶이
#### 재료
- 떡볶이떡 150g
- 당근 1/2개
- 파 1뿌리
- 양파 1/2개
- 양배추 1/4통
- 어묵 2장
- 오징어
- 새우
- 조갯살
- 홍합살

#### 양념장
- 고춧가루 2T
- 고추장 1T
- 물엿 1T
- 간장 1T
- 참기름 1T
- 다진 마늘 1/2T
- 깨 1/2T
- 후추가루 약간

#### 요리 방법
1. 떡볶이떡은 끓는 물에 살짝 데쳐 줍니다.
2. 당근, 파, 양파, 양배추, 어묵을 먹기 좋은 크기로 썰어줍니다.
3. 오징어, 새우, 조갯살, 홍합살은 깨끗이 씻어서 준비합니다.
4. 큰 팬에 참기름을 두르고 다진 마늘을 볶아 향을 내줍니다.
5. 어묵과 해물을 넣고 골고루 볶아줍니다.
6. 야채를 넣고 함께 볶아줍니다.
7. 

In [ ]:
# 먹지 말아야 할 것들 뽑고: vector DB에서 될지?


# 먹어야할 것들 뽑기(top10)
# GPT한테 걸러달라고 하기 (점수 매겨달라구)
# 건강관점 10점 만점에 몇 점

In [ ]:
# MVP에는 전체 다 넣지 않아도 될 듯? 
# 1만 개



# 배포
# 먹어야할 것들 가리기
# 프롬프트